In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os

sys.path.insert(0, '../')

from global_config import config
from utils.plot_utils import *

#from statsmodels.nonparametric.smoothers_lowess import lowess

data_dir = config.get_property('covid_fb2')

dict_correct = {'Los Martires': 'Los Mártires', 'Fontibon': 'Fontibón', 'Engativa': 'Engativá',
                            'San Cristobal': 'San Cristóbal', 'Usaquen': 'Usaquén',
                            'Ciudad Bolivar': 'Ciudad Bolívar', 'Candelaria': 'La Candelaria'}

In [15]:
from datetime import datetime, timedelta


data_dir_mnps = config.get_property('geo_dir')
data_dir      = config.get_property('data_dir')
results_dir   = config.get_property('results_dir')

path_to_save = '/Users/chaosdonkey06/Dropbox/SantosVega-Lab/RotatingLockdowns/bogota_rotating_lockdowns'

df_bogota = pd.read_csv(os.path.join(path_to_save, "data", 'cases', 'cases_prepared.csv'), index_col=None).rename(columns={'date_time': 'date'})
df_bogota['poly_id']   = df_bogota['region'].map(lambda x: x.split('-')[0].strip() )
df_bogota['poly_name'] = df_bogota['region'].map(lambda x: x.split('-')[-1].strip())

path_to_save = '/Users/chaosdonkey06/Dropbox/SantosVega-Lab/RotatingLockdowns/bogota_rotating_lockdowns'
poly_ids     = [ '{0:02d}'.format(n) for n in range(1,20)]

g1 = ['Chapinero', 'Los Mártires', 'San Cristóbal', 'Tunjuelito',
    'Rafael Uribe Uribe', 'Ciudad Bolívar', 'Santa Fe', 'Usme']
g2 = ['Bosa', 'Antonio Nariño', 'Kennedy', 'Puente Aranda', 'Fontibón']
g3 = ['Suba', 'Engativá', 'Barrios Unidos']
g4 = ['Usaquén', 'Chapinero', 'Santa Fe', 'La Candelaria', 'Teusaquillo', 'Puente Aranda', 'Antonio Nariño']

all_localities = g1+g2+g3+g4

rt_df_all = []
for idx, l in enumerate(np.unique(all_localities)):
    df_bogota_loc = df_bogota[df_bogota.poly_name==l]
    poly_id       = df_bogota_loc['poly_id'].iloc[0]

    rt_df = pd.read_csv(os.path.join(path_to_save, 'rt', 'rt_df_{}_confirmation.csv'.format(poly_id)))
    rt_df['region']    = l
    rt_df['region_id'] = poly_id
    rt_df_all.append(rt_df)
rt_df_all = pd.concat(rt_df_all)
rt_df_all = rt_df_all[rt_df_all.type=='estimate']
rt_df_all = rt_df_all[rt_df_all.variable=='R']

rt_df_all['date'] = rt_df_all['date'].map(lambda x: pd.to_datetime(0)+timedelta(days=x))
rt_df_all = rt_df_all[['date', 'median', 'mean', 'lower_90', 'lower_50', 'lower_20', 'upper_20', 'upper_50', 'upper_90', 'region', 'region_id']]


In [ ]:
# Defines lockdowns
no_lock_lab = "No Lockdown Imposed"
lock_lab    = "Lockdown Imposed"


lockdowns = []
lockdowns.append({"code": "A",
                  "start_date" : pd.to_datetime("2020-07-13"),
                  "end_date"   : pd.to_datetime("2020-07-23"),
                  "places":["chapinero",
                            "los_martires",
                            "san_cristobal",
                            "tunjuelito",
                            "rafel_uribe_uribe",
                            "ciudad_bolivar",
                            "santa_fe",
                            "usme"]})
lockdowns.append({"code": "B",
                  "start_date" : pd.to_datetime("2020-07-23"),
                  "end_date"   : pd.to_datetime("2020-08-06"),
                  "places":["bosa",
                            "antonio_narino",
                            "kennedy",
                            "puente_aranda",
                            "fontibon"]})

lockdowns.append({"code": "C",
                  "start_date" : pd.to_datetime("2020-07-31"),
                  "end_date"   : pd.to_datetime("2020-08-14"),
                  "places":["suba",
                            "engativa",
                            "barrios_unidos"]})

lockdowns.append({
                  "code": "D",
                  "start_date" : pd.to_datetime("2020-08-16"),
                  "end_date"   : pd.to_datetime("2020-08-27"),
                  "places":["usaquen",
                            "chapinero",
                            "santa_fe",
                            "candelaria",
                            "teusaquillo",
                           "puente_aranda",
                           "antonio_narino"]})


In [18]:
# prelockdown period
pre_lockdown_start = pd.to_datetime("2020-06-20")
pre_lockdown_end   = pd.to_datetime("2020-07-12")

lockdown_start     = pd.to_datetime("2020-07-13")
lockdown_end       = pd.to_datetime("2020-08-27")

# poslockdown period
pos_lockdown_start = pd.to_datetime("2020-08-28")
pos_lockdown_end   = pd.to_datetime("2020-09-15")

# Assigns moments
rt_df_all.loc[(rt_df_all.date <= pre_lockdown_end)
            & (rt_df_all.date >= pre_lockdown_start),'lockdown_type'] = "pre-lockdown1"

rt_df_all.loc[(rt_df_all.date <= lockdown_end)
            & (rt_df_all.date >= lockdown_start),'lockdown_type'] = "lockdown1"

rt_df_all.loc[(rt_df_all.date <= pos_lockdown_end)
            & (rt_df_all.date >= pos_lockdown_start),'lockdown_type'] = "pos-lockdown1"

rt_df_all["lockdown_group"] = 'None'
rt_df_all["locked_down"]    = False

for lock_info in lockdowns:
    g_lockdown_start     = lock_info["start_date"]
    g_lockdown_end       = lock_info["end_date"]
    rt_df_all.loc[ (rt_df_all.date <= g_lockdown_end)
                    & (rt_df_all.date >= g_lockdown_start), 'lockdown_group'] += "-"+lock_info["code"]


    rt_df_all.loc[ (rt_df_all.date <= g_lockdown_end)
                    & (rt_df_all.date >= g_lockdown_start) & (rt_df_all.location_id.isin(lock_info["places"]) ), 'locked_down'] = True

rt_df_all["lockdown_group"] = rt_df_all["lockdown_group"].map(lambda x: x.split('-')[-1])
rt_pivoted_loc_df = pd.pivot_table(rt_df_all, values='median', index=['date', 'lockdown_type', 'lockdown_group', "locked_down"], columns=['location_id']).fillna(0).reset_index()
rt_loc_df         = pd.pivot_table(rt_df_all, values='median', index=['date', 'location_id', 'lockdown_type', 'lockdown_group', "locked_down"], columns=['variable']).fillna(0).reset_index()


AttributeError: 'DataFrame' object has no attribute 'location_id'